<h1> library </h1>

In [1]:
import pandas as pd
import numpy as np
import os
import torch
import cv2 as cv
import tensorflow
import tensorboard
import random

from detectron2 import model_zoo
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

import warnings
warnings.filterwarnings("ignore")

In [2]:
torch.cuda.is_available()

True

<h1> importing annotation </h1>

In [3]:
path_train_annotation = 'dataset/annotations/wider_face_train_bbx_gt.txt'
path_val_annotation = 'dataset/annotations/wider_face_val_bbx_gt.txt'

train = pd.read_csv(path_train_annotation,header=None)
val = pd.read_csv(path_val_annotation,header=None)

<h1> reformating to dataframe </h1>

In [4]:
def reformat(df):
    values = df[0].values
    
    names=[]
    num_faces=[]
    bbox=[]
    
    for i in range(len(values)):
        if ".jpg" in values[i]:
            num=int(values[i+1])
            names.append(values[i])
            num_faces.append(num)
            
            box=[]
            for j in range(i+2,i+2+num):
                box.append(values[j])
            
            bbox.append(box)
        
    return pd.DataFrame({'name':names,
                         'num_faces':num_faces,
                         'bbox':bbox})

In [5]:
train = reformat(train)
val = reformat(val)

In [6]:
train.head()

,name,num_faces,bbox
0,0--Parade/0_Parade_marchingband_1_849.jpg,1,[449 330 122 149 0 0 0 0 0 0 ]
1,0--Parade/0_Parade_Parade_0_904.jpg,1,[361 98 263 339 0 0 0 0 0 0 ]
2,0--Parade/0_Parade_marchingband_1_799.jpg,21,"[78 221 7 8 2 0 0 0 0 0 , 78 238 14 17 2 0 0 0..."
3,0--Parade/0_Parade_marchingband_1_117.jpg,9,"[69 359 50 36 1 0 0 0 0 1 , 227 382 56 43 1 0 ..."
4,0--Parade/0_Parade_marchingband_1_778.jpg,35,"[27 226 33 36 1 0 0 0 2 0 , 63 95 16 19 2 0 0 ..."


In [7]:
val.head()

,name,num_faces,bbox
0,0--Parade/0_Parade_marchingband_1_465.jpg,126,"[345 211 4 4 2 0 0 0 2 0 , 331 126 3 3 0 0 0 1..."
1,0--Parade/0_Parade_Parade_0_628.jpg,29,"[26 299 10 16 2 0 0 0 2 0 , 25 329 7 11 2 0 0 ..."
2,0--Parade/0_Parade_marchingband_1_765.jpg,132,"[311 131 8 9 1 0 0 0 0 0 , 299 143 10 11 1 0 0..."
3,0--Parade/0_Parade_Parade_0_194.jpg,5,"[111 425 122 127 0 1 0 0 0 1 , 209 347 70 103 ..."
4,0--Parade/0_Parade_marchingband_1_379.jpg,26,"[281 303 20 36 2 0 0 0 2 0 , 260 324 16 21 2 0..."


<h1> Preprocessing Data Train </h1>

In [8]:
for i in train['name']:
    name_jpg = str(i).rsplit('/')[1]
    train['name'] = train['name'].replace({i:name_jpg})

In [9]:
train.head()

,name,num_faces,bbox
0,0_Parade_marchingband_1_849.jpg,1,[449 330 122 149 0 0 0 0 0 0 ]
1,0_Parade_Parade_0_904.jpg,1,[361 98 263 339 0 0 0 0 0 0 ]
2,0_Parade_marchingband_1_799.jpg,21,"[78 221 7 8 2 0 0 0 0 0 , 78 238 14 17 2 0 0 0..."
3,0_Parade_marchingband_1_117.jpg,9,"[69 359 50 36 1 0 0 0 0 1 , 227 382 56 43 1 0 ..."
4,0_Parade_marchingband_1_778.jpg,35,"[27 226 33 36 1 0 0 0 2 0 , 63 95 16 19 2 0 0 ..."


In [10]:
for i in os.listdir("dataset/train/"):
    train.loc[train['name'] == i, 'name'] = 'dataset/train/'+i

In [11]:
train.head()

,name,num_faces,bbox
0,0_Parade_marchingband_1_849.jpg,1,[449 330 122 149 0 0 0 0 0 0 ]
1,0_Parade_Parade_0_904.jpg,1,[361 98 263 339 0 0 0 0 0 0 ]
2,0_Parade_marchingband_1_799.jpg,21,"[78 221 7 8 2 0 0 0 0 0 , 78 238 14 17 2 0 0 0..."
3,dataset/train/0_Parade_marchingband_1_117.jpg,9,"[69 359 50 36 1 0 0 0 0 1 , 227 382 56 43 1 0 ..."
4,0_Parade_marchingband_1_778.jpg,35,"[27 226 33 36 1 0 0 0 2 0 , 63 95 16 19 2 0 0 ..."


In [12]:
train['mark'] = train['name'].str.contains('dataset')
train.head()

,name,num_faces,bbox,mark
0,0_Parade_marchingband_1_849.jpg,1,[449 330 122 149 0 0 0 0 0 0 ],False
1,0_Parade_Parade_0_904.jpg,1,[361 98 263 339 0 0 0 0 0 0 ],False
2,0_Parade_marchingband_1_799.jpg,21,"[78 221 7 8 2 0 0 0 0 0 , 78 238 14 17 2 0 0 0...",False
3,dataset/train/0_Parade_marchingband_1_117.jpg,9,"[69 359 50 36 1 0 0 0 0 1 , 227 382 56 43 1 0 ...",True
4,0_Parade_marchingband_1_778.jpg,35,"[27 226 33 36 1 0 0 0 2 0 , 63 95 16 19 2 0 0 ...",False


In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12880 entries, 0 to 12879
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       12880 non-null  object
 1   num_faces  12880 non-null  int64 
 2   bbox       12880 non-null  object
 3   mark       12880 non-null  bool  
dtypes: bool(1), int64(1), object(2)
memory usage: 314.6+ KB


In [14]:
train = train[train['mark'] == True]
train.head()

,name,num_faces,bbox,mark
3,dataset/train/0_Parade_marchingband_1_117.jpg,9,"[69 359 50 36 1 0 0 0 0 1 , 227 382 56 43 1 0 ...",True
5,dataset/train/0_Parade_Parade_0_343.jpg,18,"[134 580 12 14 2 0 0 0 1 0 , 55 581 5 10 2 0 1...",True
6,dataset/train/0_Parade_marchingband_1_205.jpg,65,"[60 56 19 19 2 0 1 0 1 0 , 39 2 17 21 2 0 1 0 ...",True
7,dataset/train/0_Parade_Parade_0_106.jpg,40,"[9 260 18 23 1 0 0 0 0 0 , 61 248 24 29 0 0 0 ...",True
8,dataset/train/0_Parade_Parade_0_476.jpg,35,"[566 158 22 17 2 0 0 0 2 0 , 663 118 24 28 2 0...",True


In [15]:
train.drop('mark', axis=1, inplace=True)
train.reset_index(drop=True, inplace=True)

In [16]:
train.head()

,name,num_faces,bbox
0,dataset/train/0_Parade_marchingband_1_117.jpg,9,"[69 359 50 36 1 0 0 0 0 1 , 227 382 56 43 1 0 ..."
1,dataset/train/0_Parade_Parade_0_343.jpg,18,"[134 580 12 14 2 0 0 0 1 0 , 55 581 5 10 2 0 1..."
2,dataset/train/0_Parade_marchingband_1_205.jpg,65,"[60 56 19 19 2 0 1 0 1 0 , 39 2 17 21 2 0 1 0 ..."
3,dataset/train/0_Parade_Parade_0_106.jpg,40,"[9 260 18 23 1 0 0 0 0 0 , 61 248 24 29 0 0 0 ..."
4,dataset/train/0_Parade_Parade_0_476.jpg,35,"[566 158 22 17 2 0 0 0 2 0 , 663 118 24 28 2 0..."


In [17]:
train['bbox'] = train['bbox'].apply(lambda row:[np.float_(annos.split()[:4]) for annos in row])

In [18]:
train.head()

,name,num_faces,bbox
0,dataset/train/0_Parade_marchingband_1_117.jpg,9,"[[69.0, 359.0, 50.0, 36.0], [227.0, 382.0, 56...."
1,dataset/train/0_Parade_Parade_0_343.jpg,18,"[[134.0, 580.0, 12.0, 14.0], [55.0, 581.0, 5.0..."
2,dataset/train/0_Parade_marchingband_1_205.jpg,65,"[[60.0, 56.0, 19.0, 19.0], [39.0, 2.0, 17.0, 2..."
3,dataset/train/0_Parade_Parade_0_106.jpg,40,"[[9.0, 260.0, 18.0, 23.0], [61.0, 248.0, 24.0,..."
4,dataset/train/0_Parade_Parade_0_476.jpg,35,"[[566.0, 158.0, 22.0, 17.0], [663.0, 118.0, 24..."


In [19]:
# creating empty dataframes
train_df= pd.DataFrame()

In [20]:
train_df = pd.concat([train_df, train])

In [21]:
train_df.shape

(1000, 3)

In [22]:
# custom annotation format
idx=0
values = train_df.values[idx]
print(values)

['dataset/train/0_Parade_marchingband_1_117.jpg' 9
 list([array([ 69., 359.,  50.,  36.]), array([227., 382.,  56.,  43.]), array([296., 305.,  44.,  26.]), array([353., 280.,  40.,  36.]), array([885., 377.,  63.,  41.]), array([819., 391.,  34.,  43.]), array([727., 342.,  37.,  31.]), array([598., 246.,  33.,  29.]), array([740., 308.,  45.,  33.])])]


In [23]:
# # create annotation dict 
# record = {}
# # image name        
# filename = values[0]
# # height and width of an image
# height, width = cv.imread(filename).shape[:2]
# # creating fields    
# record["file_name"] = filename
# record["image_id"] = 0
# record["height"] = height
# record["width"] = width

In [24]:
# # create bbox list
# objs = []
  
# # for every face in an image
# for i in range(len(values[2])):
    
#   # fetch bbox coordinates
#     annos = values[2][i]
    
#     # unpack values
#     x1, y1, w, h = annos[0], annos[1], annos[2], annos[3]
  
#   # find bottom right corner
#     x2, y2 = x1 + w, y1 + h
  
#   # create bbox dict
#     obj = {     "bbox": [x1, y1, x2, y2],
#                 "bbox_mode": BoxMode.XYXY_ABS,
#                 "category_id": 0,
#                 "iscrowd": 0
#         }
#   # append bbox dict to bbox list
#     objs.append(obj)

# # assigning bbox list to annotation dict
# record["annotations"] = objs
# # standard annotation format
# record

In [25]:
def create_annotation(df):  
  
  # create list to store annotation dict
    dataset_dicts = []
  # for each image 
    for idx, v in enumerate(df.values):
    
    # create annotation dict 
        record = {}
    # image name        
        filename = v[0]
    # height and width of an image
        height, width = cv.imread(filename).shape[:2]
    # assign values to fields    
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
        # create list for bbox
        objs = []
        for i in range(len(v[2])):
          # bounding box coordinates
            annos = v[2][i]

          # unpack values
            x1, y1, w, h = annos[0], annos[1], annos[2], annos[3]

          # find bottom right corner
            x2, y2 = x1 + w, y1 + h

          # create bbox dict
            obj = { "bbox": [x1, y1, x2, y2],
                    "bbox_mode": BoxMode.XYXY_ABS,
                    "category_id": 0,
                    "iscrowd": 0
                  }
          # append bbox dict to a bbox list      
            objs.append(obj)
    
    # assign bbox list to annotation dict
        record["annotations"] = objs
    # append annotation dict to list
        dataset_dicts.append(record)
    
    return dataset_dicts

In [26]:
# create standard annotations for training and validation datasets
train_annotation = create_annotation(train_df)

In [27]:
# standard annotation of an image
train_annotation[0]

{'file_name': 'dataset/train/0_Parade_marchingband_1_117.jpg',
 'image_id': 0,
 'height': 682,
 'width': 1024,
 'annotations': [{'bbox': [69.0, 359.0, 119.0, 395.0],
   'bbox_mode': <BoxMode.XYXY_ABS: 0>,
   'category_id': 0,
   'iscrowd': 0},
  {'bbox': [227.0, 382.0, 283.0, 425.0],
   'bbox_mode': <BoxMode.XYXY_ABS: 0>,
   'category_id': 0,
   'iscrowd': 0},
  {'bbox': [296.0, 305.0, 340.0, 331.0],
   'bbox_mode': <BoxMode.XYXY_ABS: 0>,
   'category_id': 0,
   'iscrowd': 0},
  {'bbox': [353.0, 280.0, 393.0, 316.0],
   'bbox_mode': <BoxMode.XYXY_ABS: 0>,
   'category_id': 0,
   'iscrowd': 0},
  {'bbox': [885.0, 377.0, 948.0, 418.0],
   'bbox_mode': <BoxMode.XYXY_ABS: 0>,
   'category_id': 0,
   'iscrowd': 0},
  {'bbox': [819.0, 391.0, 853.0, 434.0],
   'bbox_mode': <BoxMode.XYXY_ABS: 0>,
   'category_id': 0,
   'iscrowd': 0},
  {'bbox': [727.0, 342.0, 764.0, 373.0],
   'bbox_mode': <BoxMode.XYXY_ABS: 0>,
   'category_id': 0,
   'iscrowd': 0},
  {'bbox': [598.0, 246.0, 631.0, 275.0],
 

In [28]:
# register dataset
DatasetCatalog.register("face_train", lambda d="train": create_annotation(train_df))

In [29]:
# register metadata
MetadataCatalog.get("face_train").set(thing_classes=["face"])

namespace(name='face_train', thing_classes=['face'])

In [30]:
# for drawing bounding boxes on images
from detectron2.utils.visualizer import Visualizer
# for displaying an image
# from google.colab.patches import cv2_imshow
# for randomly selecting images
import random
# get the name of the classes
face_metadata = MetadataCatalog.get("face_train")
print(face_metadata)

Metadata(name='face_train', thing_classes=['face'])


In [31]:
# # randomly select images
# for d in random.sample(train_annotation, 5):
  
#   # read an image
#     img = cv.imread(d["file_name"])
#   # create visualizer
#     visualizer = Visualizer(img[:, :, ::-1], metadata=face_metadata, scale=0.5)
  
#   # draw bounding box on image
#     vis = visualizer.draw_dataset_dict(d)
  
#   # display an image
#     cv.imshow(vis.get_image()[:, :, ::-1])

<h1> Config for Fine Tunning </h1>

In [32]:
cfg = get_cfg()

In [33]:
# Get a model specified by relative path under Detectron2’s official configs/ directory.
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_50_FPN_1x.yaml"))

Loading config c:\users\asus\detectron2\detectron2\model_zoo\configs\COCO-Detection\../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


In [34]:
# load pretrained weights
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/retinanet_R_50_FPN_1x.yaml")

In [35]:
# List of the dataset names for training. Must be registered in DatasetCatalog
cfg.DATASETS.TRAIN = ("face_train",)
cfg.DATASETS.TEST = ()

In [36]:
# no. of images per batch
cfg.SOLVER.IMS_PER_BATCH = 1
# set base learning rate
cfg.SOLVER.BASE_LR = 0.001
# no. of iterations 
cfg.SOLVER.MAX_ITER = 20
# only has one class (face)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

cfg.MODEL.DEVICE='cpu'

In [37]:
# create directory to save weights
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

<h1> Training Model </h1>

In [38]:
# create a trainer with given config
trainer = DefaultTrainer(cfg)

[01/06 17:59:38 d2.engine.defaults]: Model:
RetinaNet(
  (backbone): FPN(
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelP6P7(
      (p6): Conv2d(2048, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2)

In [39]:
# If resume==True, and last checkpoint exists, resume from it, load all checkpointables (eg. optimizer and scheduler) and update iteration counter.
# Otherwise, load the model specified by the config (skip all checkpointables) and start from the first iteration.
trainer.resume_or_load(resume=False)

[01/06 17:59:47 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-Detection/retinanet_R_50_FPN_1x/190397773/model_final_bfca0b.pkl ...


The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


In [40]:
train.shape

(1000, 3)

In [41]:
# train the model
trainer.train()

[01/06 17:36:26 d2.engine.train_loop]: Starting training from iteration 0
[01/06 17:39:32 d2.utils.events]:  eta: 0:00:00  iter: 19  total_loss: 1.31  loss_cls: 0.6804  loss_box_reg: 0.4486  time: 8.7216  data_time: 0.3879  lr: 0.00095005  max_mem: 0M
[01/06 17:39:32 d2.engine.hooks]: Overall training speed: 18 iterations in 0:02:36 (8.7217 s / it)
[01/06 17:39:32 d2.engine.hooks]: Total training time: 0:02:37 (0:00:00 on hooks)


<h1> Preprocessing Data Test </h1>

In [41]:
for i in val['name']:
    name_jpg = str(i).rsplit('/')[1]
    val['name'] = val['name'].replace({i:name_jpg})

In [42]:
for i in os.listdir("dataset/val/"):
    val.loc[val['name'] == i, 'name'] = 'dataset/val/'+i

In [43]:
val['mark'] = val['name'].str.contains('dataset')
val.head(10)

,name,num_faces,bbox,mark
0,0_Parade_marchingband_1_465.jpg,126,"[345 211 4 4 2 0 0 0 2 0 , 331 126 3 3 0 0 0 1...",False
1,0_Parade_Parade_0_628.jpg,29,"[26 299 10 16 2 0 0 0 2 0 , 25 329 7 11 2 0 0 ...",False
2,0_Parade_marchingband_1_765.jpg,132,"[311 131 8 9 1 0 0 0 0 0 , 299 143 10 11 1 0 0...",False
3,0_Parade_Parade_0_194.jpg,5,"[111 425 122 127 0 1 0 0 0 1 , 209 347 70 103 ...",False
4,0_Parade_marchingband_1_379.jpg,26,"[281 303 20 36 2 0 0 0 2 0 , 260 324 16 21 2 0...",False
5,0_Parade_Parade_0_814.jpg,36,"[74 417 8 9 2 0 0 0 1 0 , 54 394 4 6 0 0 0 1 0...",False
6,0_Parade_Parade_0_470.jpg,25,"[3 152 34 51 1 0 0 0 1 0 , 36 166 85 104 0 0 0...",False
7,0_Parade_marchingband_1_1045.jpg,22,"[1 314 12 24 2 0 0 0 1 0 , 90 273 33 33 2 0 0 ...",False
8,0_Parade_marchingband_1_556.jpg,13,"[82 262 23 28 2 0 0 0 1 0 , 147 287 35 43 2 0 ...",False
9,0_Parade_Parade_0_829.jpg,1,[501 160 285 443 0 0 0 0 0 0 ],False


In [44]:
val = val[val['mark'] == True]
val.head()

,name,num_faces,bbox,mark
25,dataset/val/0_Parade_marchingband_1_74.jpg,130,"[190 362 34 45 1 0 0 0 0 0 , 294 321 13 14 2 0...",True
26,dataset/val/0_Parade_marchingband_1_234.jpg,54,"[188 328 16 15 1 0 0 0 1 0 , 216 294 17 19 0 0...",True
33,dataset/val/0_Parade_marchingband_1_309.jpg,15,"[47 90 24 24 2 0 0 0 1 0 , 127 119 26 24 2 0 0...",True
37,dataset/val/0_Parade_marchingband_1_188.jpg,47,"[252 503 11 14 1 0 0 0 0 0 , 210 417 10 10 2 0...",True
40,dataset/val/0_Parade_marchingband_1_20.jpg,22,"[29 401 29 36 2 0 0 0 0 0 , 255 369 32 39 1 0 ...",True


In [46]:
val.drop('mark', axis=1, inplace=True)
val.reset_index(drop=True, inplace=True)

In [47]:
val['bbox'] = val['bbox'].apply(lambda row:[np.float_(annos.split()) for annos in row])

In [48]:
val_df = pd.DataFrame()

In [49]:
val_df = pd.concat([val_df, val])

In [50]:
val_df

,name,num_faces,bbox
0,dataset/val/0_Parade_marchingband_1_149.jpg,24,"[[213.0, 233.0, 21.0, 32.0, 1.0, 0.0, 0.0, 0.0..."
1,dataset/val/20_Family_Group_Family_Group_20_48...,6,"[[121.0, 427.0, 77.0, 98.0, 0.0, 0.0, 0.0, 0.0..."
2,dataset/val/20_Family_Group_Family_Group_20_67...,67,"[[110.0, 951.0, 24.0, 27.0, 2.0, 0.0, 0.0, 0.0..."
3,dataset/val/20_Family_Group_Family_Group_20_64...,5,"[[106.0, 498.0, 118.0, 146.0, 0.0, 0.0, 0.0, 0..."
4,dataset/val/20_Family_Group_Family_Group_20_69...,7,"[[111.0, 391.0, 83.0, 108.0, 1.0, 0.0, 0.0, 0...."
5,dataset/val/20_Family_Group_Family_Group_20_63...,18,"[[193.0, 131.0, 14.0, 23.0, 1.0, 0.0, 0.0, 0.0..."
6,dataset/val/20_Family_Group_Family_Group_20_59...,5,"[[391.0, 272.0, 89.0, 93.0, 0.0, 0.0, 0.0, 0.0..."
7,dataset/val/20_Family_Group_Family_Group_20_54...,29,"[[47.0, 280.0, 37.0, 43.0, 1.0, 0.0, 0.0, 0.0,..."
8,dataset/val/20_Family_Group_Family_Group_20_66...,7,"[[781.0, 261.0, 37.0, 48.0, 0.0, 0.0, 0.0, 0.0..."
9,dataset/val/20_Family_Group_Family_Group_20_49...,11,"[[18.0, 188.0, 45.0, 48.0, 0.0, 0.0, 0.0, 0.0,..."


In [51]:
val_annotation = create_annotation(val_df)

In [52]:
val_annotation[0]

{'file_name': 'dataset/val/0_Parade_marchingband_1_149.jpg',
 'image_id': 0,
 'height': 684,
 'width': 1024,
 'annotations': [{'bbox': [213.0, 233.0, 234.0, 265.0],
   'bbox_mode': <BoxMode.XYXY_ABS: 0>,
   'category_id': 0,
   'iscrowd': 0},
  {'bbox': [278.0, 228.0, 305.0, 261.0],
   'bbox_mode': <BoxMode.XYXY_ABS: 0>,
   'category_id': 0,
   'iscrowd': 0},
  {'bbox': [351.0, 235.0, 377.0, 269.0],
   'bbox_mode': <BoxMode.XYXY_ABS: 0>,
   'category_id': 0,
   'iscrowd': 0},
  {'bbox': [405.0, 221.0, 427.0, 254.0],
   'bbox_mode': <BoxMode.XYXY_ABS: 0>,
   'category_id': 0,
   'iscrowd': 0},
  {'bbox': [473.0, 231.0, 497.0, 263.0],
   'bbox_mode': <BoxMode.XYXY_ABS: 0>,
   'category_id': 0,
   'iscrowd': 0},
  {'bbox': [463.0, 304.0, 488.0, 336.0],
   'bbox_mode': <BoxMode.XYXY_ABS: 0>,
   'category_id': 0,
   'iscrowd': 0},
  {'bbox': [389.0, 305.0, 414.0, 341.0],
   'bbox_mode': <BoxMode.XYXY_ABS: 0>,
   'category_id': 0,
   'iscrowd': 0},
  {'bbox': [309.0, 312.0, 334.0, 346.0],
  

In [53]:
# register validation dataset
DatasetCatalog.register("face_val", lambda d="val": create_annotation(val_df))

In [54]:
# register metadata
MetadataCatalog.get("face_val").set(thing_classes=["face"])

namespace(name='face_val', thing_classes=['face'])

In [55]:
# load the final weights
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

In [56]:
# set the testing threshold for this model
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.8

In [57]:
# List of the dataset names for validation. Must be registered in DatasetCatalog
cfg.DATASETS.TEST = ("face_val",)

In [58]:
# set up predictor
from detectron2.engine import DefaultPredictor
# Create a simple end-to-end predictor with the given config that runs on single device for a single input image.
predictor = DefaultPredictor(cfg)

[01/06 17:39:55 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output\model_final.pth ...


In [59]:
# create standard annotations for validation data
dataset_dicts = create_annotation(val_df)

In [60]:
# # randomly select images
# for d in random.sample(dataset_dicts,3):    
    
#     # read an image
#     im = cv.imread(d["file_name"])
    
#     # make predictions
#     outputs = predictor(im)
    
#     # create visualizer
#     v = Visualizer(im[:, :, ::-1], metadata=face_metadata, scale=0.5)
    
#     # draw predictions on the image
#     v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    
#     # display image
#     # cv2_imshow(v.get_image()[:, :, ::-1])
# cv.imshow('random result', v.ge)
# cv.waitKey(0)
# # print(outputs)

In [61]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

In [62]:
# create a evaluator using COCO metrics
evaluator = COCOEvaluator("face_val", cfg, False, output_dir="./output/")

[01/06 17:39:56 d2.evaluation.coco_evaluation]: Fast COCO eval is not built. Falling back to official COCO eval.
WARNING [01/06 17:39:56 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[01/06 17:39:56 d2.evaluation.coco_evaluation]: Trying to convert 'face_val' to COCO format ...
WARNING [01/06 17:39:56 d2.data.datasets.coco]: Using previously cached COCO format annotations at './output/face_val_coco_format.json'. You need to clear the cache file if your dataset has been modified.


In [63]:
# create a loader for test data
val_loader = build_detection_test_loader(cfg, "face_val")

[01/06 17:39:56 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
|    face    | 179          |
|            |              |
[01/06 17:39:56 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[01/06 17:39:56 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common.NumpySerializedList'>
[01/06 17:39:56 d2.data.common]: Serializing 10 elements to byte tensors and concatenating them all ...
[01/06 17:39:56 d2.data.common]: Serialized dataset takes 0.02 MiB


In [65]:
# runs the model on each image in the test data and produces the results
eval_results = inference_on_dataset(trainer.model, val_loader, evaluator)
eval_results

[01/06 17:40:18 d2.evaluation.evaluator]: Start inference on 10 batches


RuntimeError: DataLoader worker (pid(s) 9428, 20296, 12504, 11744) exited unexpectedly